In [ ]:
!pip install optimum[onnxruntime-gpu]

In [ ]:
!pip install transformers

In [9]:
!pip install -q datasets

In [27]:
from optimum.onnxruntime import ORTModelForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import numpy as np
from datasets import load_metric, load_dataset
import time


In [11]:
import pandas as pd
df = pd.read_csv('rubia_train.csv')


In [38]:
checkpoint = 'DeepPavlov/rubert-base-cased'

In [12]:
df

,id,text,class
0,0,все мужчины грубые,0
1,1,все миллионеры помогают бедным,1
2,2,"брак придуман для нормальных людей, а не для г...",1
3,3,она могучая строительница,0
4,4,лиза обняла свою дочь,1
...,...,...,...
3836,3836,все украинцы наглые,1
3837,3837,"у тебя мало денег, потому что ты упускаешь воз...",1
3838,3838,все шведы дворники,0
3839,3839,витя сделал игорю предложение,0


In [19]:
accuracy_score = load_metric("accuracy")


def compute_metrics(pred):
  predictions, labels = pred
  predictions = np.argmax(predictions, axis=1)
  return accuracy_score.compute(predictions=predictions, references=labels)

<ipython-input-19-175e39ac7946>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  accuracy_score = load_metric("accuracy")


In [15]:
checkpoint = "DeepPavlov/rubert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2).to("cuda")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
print("Base Model: ", model.num_parameters())


Base Model:  177854978


In [21]:

sample_input = df['text'][101]

print(df['text'][101])
print(df['class'][101])

все люди с высоким доходом страдают от алкоголизма
0


In [25]:
pipe = pipeline("text-classification", model=model, tokenizer='DeepPavlov/rubert-base-cased', device=0)

id2label = pipe.model.config.id2label
label2id = pipe.model.config.label2id

In [28]:
#WARMUP
for _ in range(10):
  _ = pipe(sample_input)

#INFERENCE
start = time.time()
for _ in range(100):
  _ = pipe(sample_input)
total_time_model = time.time() - start
print("Общее время обработки 100 запросов базовой моделью:", total_time_model)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Общее время обработки 100 запросов базовой моделью: 1.7402067184448242


In [29]:
from typing import Callable
from tqdm.notebook import tqdm


data_test_X = df['text'][::50]
data_test_y = df['class'][::50]

def show_accuracy(model: Callable):
    model_preds = []
    for i in tqdm(data_test_X):
        model_preds.append(label2id[model(i)[0]['label']])

    print(accuracy_score.compute(predictions=model_preds, references=data_test_y))

In [30]:
show_accuracy(pipe)


  0%|          | 0/77 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


{'accuracy': 0.5324675324675324}


In [31]:
model.save_pretrained("transformersbook/bert-base-uncased-finetuned-clinc")


In [32]:
from optimum.pipelines import pipeline
from optimum.onnxruntime import ORTOptimizer, ORTModelForSequenceClassification
from optimum.onnxruntime.configuration import OptimizationConfig

In [36]:
optimum_model1 = pipeline(
        task="text-classification",
        model=checkpoint,
        tokenizer='DeepPavlov/rubert-base-cased',
        accelerator="ort",
        device=0
)

Framework not specified. Using pt to export to ONNX.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the export variant default. Available variants are:
	- default: The default ONNX variant.
Using framework PyTorch: 2.1.0+cu118
Overriding 1 configuration item(s)
	- use_cache -> False
use_io_binding was set to False, setting it to True because it can provide a huge speedup on GPUs. It is possible to disable this feature manually by setting the use_io_binding attribute back to False.


In [37]:
#WARMUP
for _ in range(10):
  _ = optimum_model1(sample_input)

#INFERENCE
start = time.time()
for _ in range(100):
  _ = optimum_model1(sample_input)
total_time_model = time.time() - start

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [39]:
model_id = checkpoint
save_dir = "optimized_model"

tokenizer = AutoTokenizer.from_pretrained(model_id)
optimum_model2 = ORTModelForSequenceClassification.from_pretrained(
    model_id,
    export=True,
    provider="CUDAExecutionProvider",
)

Framework not specified. Using pt to export to ONNX.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the export variant default. Available variants are:
	- default: The default ONNX variant.
Using framework PyTorch: 2.1.0+cu118
Overriding 1 configuration item(s)
	- use_cache -> False


In [40]:
# Load the optimization configuration detailing the optimization we wish to apply
# optimization_config = AutoOptimizationConfig.O3()
optimization_config = OptimizationConfig(
    # optimize_for_gpu=True,
    optimization_level=99
)
optimizer = ORTOptimizer.from_pretrained(optimum_model2)

optimizer.optimize(save_dir=save_dir, optimization_config=optimization_config)
# Load the optimized model from a local repository
optimized_model1 = ORTModelForSequenceClassification.from_pretrained(save_dir)

/usr/local/lib/python3.10/dist-packages/optimum/onnxruntime/configuration.py:765: FutureWarning: disable_embed_layer_norm will be deprecated soon, use disable_embed_layer_norm_fusion instead, disable_embed_layer_norm_fusion is set to True.
  warnings.warn(
Optimizing model...
Configuration saved in optimized_model/ort_config.json
Optimized model saved at: optimized_model (external data format: False; saved all tensor to one file: True)


In [41]:
# Create the transformers pipeline
onnx_clx = pipeline("text-classification", model=optimized_model1, accelerator="ort", device=0)


use_io_binding was set to False, setting it to True because it can provide a huge speedup on GPUs. It is possible to disable this feature manually by setting the use_io_binding attribute back to False.


In [44]:
#WARMUP
for _ in range(10):
  _ = onnx_clx(sample_input)

#INFERENCE
start = time.time()
for _ in range(100):
  _ = onnx_clx(sample_input)
total_time_model = time.time() - start
print("Общее время обработки 100 запросов опт. моделью:", total_time_model)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Общее время обработки 100 запросов опт. моделью: 0.7591598033905029


In [45]:
show_accuracy(onnx_clx)


  0%|          | 0/77 [00:00<?, ?it/s]

{'accuracy': 0.4935064935064935}
